In [1]:
import itertools # 是 Python 的內建模組，提供了一組用於處理迭代器的函數和工具。
                 # 它包含了各種用於高效處理迭代器的函數，可以幫助我們編寫更簡潔、高效的代碼。
import sys # 是 Python 的內建模組，提供了與 Python 解釋器和運行環境相關的功能。
# sys.path 是一個列表，包含了 Python 解釋器在導入模組時會搜尋的路徑。
# 當你使用 import 語句導入模組時 Python 會依次在 sys.path 中的路徑下尋找對應的模組文件。
sys.path.append("../ecg-classification/")
# sys.path.append("C:\\Users\\Chen_Lab01\\Documents\\GitHub/ecg-classification")
# from IPython.display import Video
# import cv2
from matplotlib import pyplot as plt
plt.style.use("ggplot") #  是 Matplotlib 庫中用於設置繪圖樣式的函數。它使用了一種名為 "ggplot" 的預定義樣式
import matplotlib
matplotlib.use("TkAgg")
                        #  該樣式模仿了 R 語言的 ggplot2 繪圖包的外觀。
# print(sys.path)
import torch
from ecg_tools.config import EcgConfig, Mode
from ecg_tools.data_loader import DatasetConfig, get_data_loaders
from ecg_tools.model import ECGformer
from ecg_tools.train import ECGClassifierTrainer


Load model

In [2]:
import torch
config = EcgConfig()    
model = torch.load("..\\..\\model_save\\model_epoch_98.pth")
model.eval()
model.to('cpu')

ECGformer(
  (encoder): ModuleList(
    (0-5): 6 x TransformerEncoderLayer(
      (0): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (1): MultiHeadAttention(
            (queries_projection): Linear(in_features=192, out_features=192, bias=True)
            (values_projection): Linear(in_features=192, out_features=192, bias=True)
            (keys_projection): Linear(in_features=192, out_features=192, bias=True)
            (final_projection): Linear(in_features=192, out_features=192, bias=True)
          )
          (2): Dropout(p=0.1, inplace=False)
        )
      )
      (1): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (1): MLP(
            (0): Linear(in_features=192, out_features=768, bias=True)
            (1): GELU(approximate='none')
            (2): Linear(in_features=768, out_features=192, bias=True)
          )
          (2):

量化模型

In [3]:
import quanto
from quanto import Calibration, freeze, qfloat8, qint4, qint8, quantize

quantize(model, weights=quanto.qint8, activations=quanto.qint8) # 量化線性層
freeze(model)
torch.save(model, "..\\..\\model_save\\model_quantized_98_quanto.pth")

準確度測試

In [4]:
import einops
loader = get_data_loaders(DatasetConfig())
accuracy = 0
model.eval()
for signal, label in loader[Mode.eval]:
    signal.to('cpu')
    label.to('cpu')
    signal = einops.rearrange(signal, "b c e -> b e c")
    # print(signal)
    p = model(signal)
    print(p)
    print(label)
    print(signal.shape, label.shape)
    print(p.argmax(1) == label)
    accuracy += torch.sum(p.argmax(1) == label)
    print(f"accuracy: {accuracy / config.dataset.batch_size}")
    break



QTensor(tensor([[  5,   1,  -5,   1,  -9,  -1],
        [  4,  -1,  -3,   1,  -5,  -2],
        [  4,  -2,  -3,   1,  -6,  -2],
        [  7,  -6,  -3,   1,  -9,   3],
        [  6,  -1,  -4,   1, -10,   0],
        [  6,   1,  -5,   0,  -9,  -1],
        [  6,   1,  -5,   0, -10,  -1],
        [  4,  -1,  -3,   1,  -6,  -2],
        [  6,   0,  -5,   1, -11,   0],
        [  4,  -1,  -2,   1,  -7,  -2],
        [  6,   0,  -5,   1, -10,   0],
        [  7,  -4,  -4,   1,  -8,   0],
        [  4,  -1,  -3,   1,  -6,  -2],
        [  4,  -1,  -2,   0,  -6,  -2],
        [  4,  -1,  -2,   0,  -6,  -2],
        [  6,   1,  -5,   1, -10,  -1],
        [  5,   0,  -5,   1, -10,   0],
        [  5,   1,  -5,   1, -10,  -1],
        [  6,   0,  -4,   1, -11,   0],
        [  6,   1,  -5,   0,  -9,  -1],
        [  6,   1,  -5,   0, -10,  -1],
        [  6,   0,  -5,   0, -10,  -1],
        [  3,  -4,  -1,   2,  -5,   2],
        [  6,  -2,  -5,   1,  -7,  -1],
        [  4,  -1,  -2,   0,  -6

## Visualize the model

Neutron 可視化

In [6]:
inputs = torch.randn(1, 187, 1)
traced_script_module = torch.jit.trace(model_quantized, inputs)
traced_script_module.save("traced_resnet_model.pth")

import netron
modelData = 'traced_resnet_model.pth'
netron.start(modelData)

NameError: name 'model_quantized' is not defined

Grapviz 可視化

In [5]:
# from torchviz import make_dot
# vis_graph = make_dot(model_quantized(inputs), params=dict(model_quantized.named_parameters()), show_attrs=True, show_saved=True)
# vis_graph.view()  # 会在当前目录下保存一个“Digraph.gv.pdf”文件，并在默认浏览器中打开

## Parameter Extraction

### 提取全部的參數(忽略量化的資訊)

In [8]:
model_state_dict = model.state_dict()

ignore_keys = [
    'scale',
    'zero_point',
    '_packed_params.dtype'
]

with open('model_layers_params.txt', 'w') as f:
    for layer_name, param_tensor in model_state_dict.items():
        if not any(ignore_key in layer_name for ignore_key in ignore_keys):
            f.write(f"Layer: {layer_name}\n") 
            f.write(f"Parameters: {str(param_tensor)}\n")
            f.write("\n")


### 提取六層 Encoder 參數

In [ ]:
model_state_dict = model_quantized.state_dict()

ignore_keys = [
    '.scale',
    '.zero_point',
    '._packed_params.dtype'
]

# Extract encoder.0 to encoder.5
for i in range(6):
    with open(f'encoder_{i}_params.txt', 'w') as f:
        for layer_name, param_tensor in model_state_dict.items():
            if layer_name.startswith(f'encoder.{i}') and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
                f.write(f"Layer: {layer_name}\n") 
                f.write(f"Parameters: {str(param_tensor)}\n")
                f.write("\n")

# Extract classifier
with open('classifier_params.txt', 'w') as f:
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith('classifier') and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
            f.write(f"Layer: {layer_name}\n")
            f.write(f"Parameters: {str(param_tensor)}\n") 
            f.write("\n")

# Extract embedding.cls_token
with open('cls_token_params.txt', 'w') as f:
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name == 'embedding.cls_token':
            f.write(f"Layer: {layer_name}\n")
            f.write(f"Parameters: {str(param_tensor)}\n")
            f.write("\n")

# Extract embedding
with open('embedding_params.txt', 'w') as f: 
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith('embedding') and layer_name != 'embedding.cls_token' and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
            f.write(f"Layer: {layer_name}\n")
            f.write(f"Parameters: {str(param_tensor)}\n")
            f.write("\n")

# Extract positional_encoding
with open('pos_encoding_params.txt', 'w') as f:
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith('positional_encoding') and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
            f.write(f"Layer: {layer_name}\n") 
            f.write(f"Parameters: {str(param_tensor)}\n")
            f.write("\n")
